In [1]:
import pandas as pd     
import re
import numpy as np 
from bs4 import BeautifulSoup 
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
wnlzer = WordNetLemmatizer()

In [ ]:
# loading our data 
train = pd.read_csv("labeledTrainData.tsv" , header = 0 ,  \
	delimiter = '\t' , quoting  = 3)
test = pd.read_csv('testData.tsv' , header = 0 , delimiter = '\t' , quoting = 3 )

##### the function takes a review in the form of a string , and after doing the processing outputs a string.
##### after that we form a list of reviews and then pass each review to the function using a for loop.  
##### in the last the list has all the reviews as individual elements of the list and is now ready to be fed into the algorithm. 

In [ ]:
def raw_string_to_list_clean_string( raw_train_review ):
	remove_html = BeautifulSoup( raw_train_review ).text
	remove_punch = re.sub('[^A-Za-z ]' , "" , remove_html)
	token = remove_punch.lower().split()
	srm_token = [wnlzer.lemmatize(i) for i in token if not i in set(stopwords.words('english'))]
	clean_text = " ".join(srm_token)
	return(clean_text)

ready_train_list = []
length  = len(train['review'])
for i in range(0 , length):
# the if loop is to track the progress only. It has nothing to do with the analysis
	if (i%100 == 0):
		print "doing  %d of  %d of training data set" % (i+1 , length)
	a = raw_string_to_list_clean_string(train['review'][i])
	ready_train_list.append(a)
    
ready_test_list = []
for i in range(0 , test_length):
	a = raw_string_to_list_clean_string(test['review'][i])
	test_text.append(a)
# if loop is to check for updates 
	if( i%100 == 0):
		print "doing    %d out of  %d  of test data " % (i+1 , test_length)



##### initially I was using the count_vectorizer but after some reading i figured out that tf-idf is better for sparse metrices 

In [ ]:
vectorizer = TfidfVectorizer(analyzer = "word" , tokenizer = None , preprocessor = None , \
	stop_words = None , max_features = 20000)
train_vectorizer = vectorizer.fit_transform(ready_train_list)
test_vectorizer  = vectorizer.transform(ready_test_list)
# i also used the hashing_vectorizer but the score was not changed much so i reverted back to tfidf vectorizer

##### my first obvious choice was logistic regression but with SGD i could use both logistic and SVM with 'loss' parameter and by using GridSearchCV .     Both logistic and SVM gave the same 78.98% . 

In [ ]:
sklearn.linear_model import SGDClassifier
clf = SGDClassifier(alpha = 0.01)
clf.fit(train_vectorizer)

In [ ]:
prediction = clf.predict(test_vectorizer)

In [ ]:
final_dataframe = pd.DataFrame( data = { 'id' : test['id'] , 'sentiment' : prediction })
final_dataframe.to_csv( "improving_on_kaggle_part01.csv" , index = False ,quoting = 3 )

Now some additional points , Apart from Hashing i also tried Truncated SVD but the Progress was too less. 